## True State VAE

Creating a simple VAE model to learn how to encode-decode true states

In [2]:
!pip install tensorflow-probability
from IPython import display

import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
import tensorflow_probability as tfp
import time

from ray.rllib.offline.json_reader import JsonReader
import numpy_indexed as npi
import pandas as pd
from true_state_viewer import TrueStateTreeGraphViz, display_tree_pairs

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:281: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [3]:
# Code from https://www.tensorflow.org/tutorials/generative/cvae
class TrueStateVAE(tf.keras.Model):
  """Convolutional variational autoencoder."""

  def __init__(self, latent_dim):
    super(TrueStateVAE, self).__init__()
    self.latent_dim = latent_dim
    self.encoder = tf.keras.Sequential(
        [
            tf.keras.layers.InputLayer(input_shape=(78)),
#             tf.keras.layers.Dense(512, activation="relu"),
#             tf.keras.layers.Dense(512, activation="relu"),
#             tf.keras.layers.Dense(8192, activation="relu"),
#             tf.keras.layers.Dense(16384, activation="relu"),
#             RUN 7
#             tf.keras.layers.Dense(10000, activation="relu"),
#             tf.keras.layers.Dense(6000, activation="relu"),
#               RUN 8 (bs32) RUN 9 (BS64)
#             tf.keras.layers.Dense(15000, activation="relu"),
#             tf.keras.layers.Dense(10000, activation="relu"),
#             RUN 10 (bs64, L8)
            tf.keras.layers.Dense(12288, activation="relu"),
            tf.keras.layers.Reshape([64,64,3]),
            tf.keras.layers.Conv2D(
                filters=64, kernel_size=3, strides=(2, 2), activation='relu'),
            tf.keras.layers.Conv2D(
                filters=128, kernel_size=3, strides=(2, 2), activation='relu'),
#             tf.keras.layers.Conv2D(
#                 filters=128, kernel_size=3, strides=(2, 2), activation='relu'),
            tf.keras.layers.Flatten(),
#             tf.keras.layers.Dense(8192, activation="relu"),
#             tf.keras.layers.Dense(4096, activation="relu"),
#             tf.keras.layers.Dense(256, activation="relu"),
#             tf.keras.layers.Dense(2048, activation="relu"),
#             tf.keras.layers.Dense(latent_dim)
            # No activation
            tf.keras.layers.Dense(latent_dim + latent_dim)
        ]
    )
#     self.dist_fc = tf.keras.Sequential(
#         [
#             tf.keras.layers.Dense(latent_dim + latent_dim)
#         ]
#     )

    self.decoder = tf.keras.Sequential(
        [
            tf.keras.layers.InputLayer(input_shape=(latent_dim,)),
#             tf.keras.layers.Dense(2048, activation="relu"),
#             tf.keras.layers.Dense(256, activation="relu"),
#             tf.keras.layers.Dense(4096, activation="relu"),
#             tf.keras.layers.Dense(8192, activation="relu"),
#             tf.keras.layers.Dense(6000, activation="relu"),
#             tf.keras.layers.Dense(10000, activation="relu"),
#             tf.keras.layers.Dense(10000, activation="relu"),
#             tf.keras.layers.Dense(15000, activation="relu"),
            tf.keras.layers.Dense(units=28800, activation=tf.nn.relu),
#             tf.keras.layers.Dense(units=4608, activation=tf.nn.relu),
#             tf.keras.layers.Reshape(target_shape=(6, 6, 128)),
            tf.keras.layers.Reshape(target_shape=(15, 15, 128)),
#             tf.keras.layers.Conv2DTranspose(
#                 filters=128, kernel_size=3, strides=2, padding='same',
#                 activation='relu'),
            tf.keras.layers.Conv2DTranspose(
                filters=128, kernel_size=3, strides=2, padding='same',
                activation='relu'),
            tf.keras.layers.Conv2DTranspose(
                filters=64, kernel_size=3, strides=2, padding='same',
                activation='relu'),
            # No activation
            tf.keras.layers.Conv2DTranspose(
                filters=1, kernel_size=3, strides=1, padding='same'),
            tf.keras.layers.Flatten(),
#             tf.keras.layers.Dense(16384, activation="relu"),
#             tf.keras.layers.Dense(8192, activation="relu"),
#             tf.keras.layers.Dense(512, activation="relu"),
#             tf.keras.layers.Dense(512, activation="relu"),
            tf.keras.layers.Dense(78)
        ]
    )

#   @tf.function
#   def get_dist(self, x):
#     mean, logvar = tf.split(self.dist_fc(x), num_or_size_splits=2, axis=1)
#     return mean, logvar

  @tf.function
  def sample(self, eps=None):
    if eps is None:
      eps = tf.random.normal(shape=(100, self.latent_dim))
    return self.decode(eps, apply_sigmoid=True)
  
  @tf.function
  def encode(self, x):
    return tf.split(self.encoder(x), num_or_size_splits=2, axis=1)

  @tf.function
  def reparameterize(self, mean, logvar):
#     eps = tf.random.normal(shape=mean.shape)
    eps = tf.random.normal(shape=tf.shape(mean))
    
    return eps * tf.exp(logvar * .5) + mean

  @tf.function
  def decode(self, z, apply_sigmoid=False):
    logits = self.decoder(z)
    if apply_sigmoid:
      probs = tf.sigmoid(logits)
      return probs
    return logits

  @tf.function
  def get_oh_output(self, predictions):
    pred_oh = tf.one_hot(tf.argmax(tf.nn.softmax(tf.reshape(predictions,(-1,2,3))),axis=2),depth=3)
    return pred_oh
    
  def call(self, inputs, training):
    mean, logvar = self.encode(inputs)
    z = self.reparameterize(mean, logvar)
    x_logit = self.decode(z)
    return x_logit
    

In [4]:
optimizer = tf.keras.optimizers.Adam(1e-4)


def log_normal_pdf(sample, mean, logvar, raxis=1):
  log2pi = tf.math.log(2. * np.pi)
  return tf.reduce_sum(
      -.5 * ((sample - mean) ** 2. * tf.exp(-logvar) + logvar + log2pi),
      axis=raxis)


# TODO: make this a combination of cat cross entropy
def compute_loss(model, x):
  mean, logvar = model.encode(x)
  z = model.reparameterize(mean, logvar)
  x_logit = model.decode(z)
  
  cross_ent = tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=x)
  logpx_z = -tf.reduce_sum(cross_ent, axis=[1])

#   THIS SHOULD WORK BETTER....
#   as suggested in https://medium.com/p/53eefdfdbcc7
#   acc = 0
#   for i in range(26):
#     cross_ent = tf.nn.softmax_cross_entropy_with_logits(logits=x_logit[:,i*3:(i*3)+3], labels=x[:,i*3:(i*3)+3])
#     acc += cross_ent#tf.reduce_sum(cross_ent, axis=[1])
#   logpx_z = -acc

#   logpx_z = logpx_z/26


  logpz = log_normal_pdf(z, 0., 0.)
  logqz_x = log_normal_pdf(z, mean, logvar)
  return -tf.reduce_mean(logpx_z + logpz - logqz_x)


@tf.function
def train_step(model, x, optimizer):
  """Executes one training step and returns the loss.

  This function computes the loss and gradients, and uses the latter to
  update the model's parameters.
  """
  with tf.GradientTape() as tape:
    loss = compute_loss(model, x)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

In [5]:
epochs = 300
# set the dimensionality of the latent space to a plane for visualization later
latent_dim = 8
# latent_dim = 5
# num_examples_to_generate = 16

# # keeping the random vector constant for generation (prediction) so
# # it will be easier to see the improvement.
# random_vector_for_generation = tf.random.normal(
#     shape=[num_examples_to_generate, latent_dim])
true_state_model = TrueStateVAE(latent_dim)

# train_size = 4445
train_test_split = 0.98
batch_size = 64

In [83]:
true_state_model.encoder.summary()

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_45 (Dense)            (None, 12288)             970752    
                                                                 
 reshape_19 (Reshape)        (None, 64, 64, 3)         0         
                                                                 
 conv2d_24 (Conv2D)          (None, 31, 31, 64)        1792      
                                                                 
 conv2d_25 (Conv2D)          (None, 15, 15, 128)       73856     
                                                                 
 flatten_20 (Flatten)        (None, 28800)             0         
                                                                 
 dense_46 (Dense)            (None, 16)                460816    
                                                                 
Total params: 1,507,216
Trainable params: 1,507,216
N

In [6]:
meander = pd.read_csv('csv_data/TrueStates_200_4000_Meander.csv')
bline = pd.read_csv('csv_data/TrueStates_1221_4000_B_Line.csv')
badbluemeander = pd.read_csv('csv_data/TrueStates_200_4000_Meander_badblue.csv')

dataset = pd.concat([meander, bline, badbluemeander], ignore_index=True)
dataset = dataset.drop_duplicates()

print(f"number of rows = {meander.shape[0]}")
print(f"number of rows = {bline.shape[0]}")
print(f"number of rows = {badbluemeander.shape[0]}")
print(f"number of rows = {dataset.shape[0]}")
# dataset=bline

number of rows = 4445
number of rows = 305
number of rows = 7972
number of rows = 12716


In [7]:
train_df=dataset.sample(frac=train_test_split,random_state=200)
test_df=dataset.drop(train_df.index)

train_size = train_df.shape[0]

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:281: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
train_dataset = (tf.data.Dataset.from_tensor_slices((train_df.iloc[:,1:].values)).shuffle(train_size).batch(batch_size))
test_dataset = (tf.data.Dataset.from_tensor_slices((test_df.iloc[:,1:].values)).shuffle(train_size).batch(1))

In [87]:

for epoch in range(1, epochs + 1):
  start_time = time.time()
  for train_x in train_dataset:
    train_x = tf.cast(train_x,tf.float32)
    train_step(true_state_model, train_x, optimizer)
  end_time = time.time()

  loss = tf.keras.metrics.Mean()
  for test_x in test_dataset:
    test_x = tf.cast(test_x,tf.float32)
    loss(compute_loss(true_state_model, test_x))
  elbo = -loss.result()
  display.clear_output(wait=False)
  print('Epoch: {}, Test set ELBO: {}, time elapse for current epoch: {}'
        .format(epoch, elbo, end_time - start_time))
#   generate_and_save_images(model, epoch, test_sample)

Epoch: 300, Test set ELBO: -11.290464401245117, time elapse for current epoch: 1.8965139389038086


In [9]:
def get_state_pred_pair(model, state):
  state = tf.cast(state, tf.float32)
  mean, logvar = model.encode(state)
  z = model.reparameterize(mean, logvar)
  predictions = model.sample(z)
    
  state_oh = tf.reshape(state,(-1,2,3)) #tf.one_hot(tf.argmax(tf.reshape(state,(-1,3)),axis=1),depth=3)
  pred_oh = model.get_oh_output(predictions)#tf.one_hot(tf.argmax(tf.nn.softmax(tf.reshape(predictions,(-1,2,3))),axis=2),depth=3)
#   print(f"{state_oh.shape}, {pred_oh.shape}")
  return state_oh, pred_oh

In [93]:
# test_dataset = (tf.data.Dataset.from_tensor_slices((dataset.iloc[:200,1:].values)).shuffle(train_size).batch(batch_size))
train_dataset2 = (tf.data.Dataset.from_tensor_slices((train_df.iloc[:,1:].values)).shuffle(train_size).batch(1))

total_matches = 0
total = 0
nodes = 0
sum_diffs_sqrd = 0
state_pred_pairs = []
state_pred_pair_tree_vis = []
for test_x in test_dataset:
    total +=1
    state_oh, pred_oh = get_state_pred_pair(true_state_model, test_x)
    
    state_pred_pairs.append([state_oh, pred_oh])
    state_pred_pair_tree_vis.append([TrueStateTreeGraphViz(state_oh), TrueStateTreeGraphViz(pred_oh)])
    
    diffs = np.rint(state_oh.numpy()) - np.rint(pred_oh.numpy())
#     diffs = get_state_diff(true_state_model,test_x)
    nodes += len(diffs.flatten())
    diffs_sqrd = np.sum(diffs*diffs)
    sum_diffs_sqrd += diffs_sqrd
    if not diffs_sqrd >0:
#       print(diffs)
#     else:
      total_matches += 1
#       print("Match")
#       print(diffs)

print(f"accuracy = {total_matches}/{total} = {total_matches/total}, \nmean of squared diffs = {sum_diffs_sqrd}/{nodes}={sum_diffs_sqrd/nodes}\npercentage wrong = ({sum_diffs_sqrd}/{2})/({nodes}/{3})={(sum_diffs_sqrd/2)/(nodes/3)}")
display_tree_pairs(state_pred_pair_tree_vis)


/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:281: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


accuracy = 163/254 = 0.6417322834645669, 
mean of squared diffs = 224.0/19812=0.011306279022814456
percentage wrong = (224.0/2)/(19812/3)=0.016959418534221685
254


/usr/local/lib/python3.8/dist-packages/ipywidgets/widgets/widget.py:443: DeprecationWarning: Passing unrecognized arguments to super(IntSlider).__init__(name='Index').
object.__init__() takes exactly one argument (the instance to initialize)
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super().__init__(**kwargs)


In [92]:
# true_state_model.build((1,78))
true_state_model(np.ones((1,78)))
true_state_model.save('models/trueStateVAE_11_L8',overwrite=True)# best one is L8 3 or 4 or 6 or 7 (78%).. need to check
# m2 = Tr
# # true_state_model.save('models/trueStateVAE_1')
# true_state_model.compile()
# # true_state_model._set_inputs(test_x)
# true_state_model.fit(dataset, epochs=1)
# tf.keras.models.save_model(
#     true_state_model,
#     'models/trueStateVAE_1',
#     overwrite=False,
#     include_optimizer=True
# )

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:281: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
m2 = tf.keras.models.load_model(
    'models/trueStateVAE_7_L8',
)

In [13]:


# m2.sample()
total_matches = 0
total = 0
nodes = 0
sum_diffs_sqrd = 0
state_pred_pairs = []
state_pred_pair_tree_vis = []
for test_x in test_dataset:
    total +=1
    state_oh, pred_oh = get_state_pred_pair(m2, test_x)
    
    state_pred_pairs.append([state_oh, pred_oh])
    state_pred_pair_tree_vis.append([TrueStateTreeGraphViz(state_oh), TrueStateTreeGraphViz(pred_oh)])
    
    diffs = np.rint(state_oh.numpy()) - np.rint(pred_oh.numpy())
#     diffs = get_state_diff(true_state_model,test_x)
    nodes += len(diffs.flatten())
    diffs_sqrd = np.sum(diffs*diffs)
    sum_diffs_sqrd += diffs_sqrd
    if not diffs_sqrd >0:
#       print(diffs)
#     else:
      total_matches += 1
#       print("Match")
#       print(diffs)

print(f"accuracy = {total_matches}/{total} = {total_matches/total}, \nmean of squared diffs = {sum_diffs_sqrd}/{nodes}={sum_diffs_sqrd/nodes}\npercentage wrong = ({sum_diffs_sqrd}/{2})/({nodes}/{3})={(sum_diffs_sqrd/2)/(nodes/3)}")
display_tree_pairs(state_pred_pair_tree_vis)

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:281: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


accuracy = 194/254 = 0.7637795275590551, 
mean of squared diffs = 134.0/19812=0.006763577629719362
percentage wrong = (134.0/2)/(19812/3)=0.010145366444579043
254


/usr/local/lib/python3.8/dist-packages/ipywidgets/widgets/widget.py:443: DeprecationWarning: Passing unrecognized arguments to super(IntSlider).__init__(name='Index').
object.__init__() takes exactly one argument (the instance to initialize)
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super().__init__(**kwargs)


In [24]:
m2.encoder.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 10000)             790000    
                                                                 
 dense_31 (Dense)            (None, 6000)              60006000  
                                                                 
 dense_32 (Dense)            (None, 16)                96016     
                                                                 
Total params: 60,892,016
Trainable params: 60,892,016
Non-trainable params: 0
_________________________________________________________________
